In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Upload CSV with only MX info

In [ ]:
df_mx = pd.read_csv('../input/youtube-new/MXvideos.csv', engine='python', encoding = 'latin1')
df_mx.info()

In [ ]:
#Change Category Title as str
df_mx['category_id'] = df_mx['category_id'].astype('str')
df_mx.info()

In [ ]:
df_mx.head()

## Cargo el json con los datos de categoria de MX

In [ ]:
import json
# leo el json
with open('../input/youtube-new/MX_category_id.json') as f:
    data = json.load(f)

* parto las categorias y el index que quiero, estos tienen forma de diccionario

In [ ]:
cats = {}
for item in data['items']:
    cats[item['id']] = item['snippet']['title']

* Transformo el el diccionario a DF para oder unirlo con el df original

In [ ]:
df_cat_mx = pd.DataFrame().from_dict(cats, orient='index')
df_cat_mx.head()

* Cambio los nombres de las columnas para hacer mas facil el merge

In [ ]:
df_cat_mx = df_cat_mx.rename_axis('category_id').reset_index()

In [ ]:
df_cat_mx.rename(columns ={0:'categoria'},inplace=True)
df_cat_mx.info()
df_cat_mx

#### Inserto una celda con category_id 29 para poder hacer elmerge y no queda vacio

In [ ]:
df_cat_mx.loc[31] = ['29','Otros']
df_cat_mx

# Merge in order to have name of categories

In [ ]:
df = pd.merge(df_mx, df_cat_mx, on = 'category_id', how = 'left')
df.info()

### Detectando si hay null en el dataset
* Solo hay null en algunas descripciones, que en este analisis no los voy a usar

In [ ]:
df.isnull().sum()

## Generate final dataset only whith features that i need

In [ ]:
# Dataset final 

df = df.drop(columns = ['trending_date','category_id','publish_time','tags','thumbnail_link','video_error_or_removed','description'])
df.head()

# Starting EDA

#### 1. Numerical features 
* 4 variables numericas
* 40k + registros
* On views 50%Q lower than mean 
* Very high desvest

In [ ]:
df.describe().round(1)

### 2. Categorical features
* video_id is unique
* Many titles are duplicate
* Most view channel Cracks
* Most view category Entertainment

In [ ]:
df.describe(include = 'object')

#### Videos by title
* Al parecer hay videos que tienen el mismo nombre pero diferente ID

In [ ]:
df.groupby(
    'title')['title'].count().sort_values(ascending = False).head(10)

### Deleting duplicates, sorted by numb of views (most viewed = lastdate)

In [ ]:
df = df.sort_values('views', ascending = False).drop_duplicates(subset = 'title')

### 3. Outlider


In [ ]:
import plotly.express as px

* The histogram show us that views are right-skewed

In [ ]:
px.histogram(df,x= 'views')

* Ploting only the first 25% quantile

In [ ]:
v_1q  = df[df['views'] <= np.quantile(df['views'],0.25)]
px.histogram(v_1q,x= 'views',nbins=90)

* Ploting only the second 25% quantile

In [ ]:
v_2q  = df[(df['views'] > np.quantile(df['views'],0.25)) & (df['views'] <= np.quantile(df['views'],0.50))]
px.histogram(v_2q,x= 'views',nbins=90)

* Ploting only the third 25% quantile

In [ ]:
v_3q  = df[(df['views'] > np.quantile(df['views'],0.50)) & (df['views'] <= np.quantile(df['views'],0.75))]
px.histogram(v_3q,x= 'views',nbins=90)

* Ploting only the last 25% quantile

In [ ]:
v_4q  = df[(df['views'] > np.quantile(df['views'],0.75)) & (df['views'] <= np.quantile(df['views'],0.99))]
px.histogram(v_4q,x= 'views',nbins=90)

* The last 25% quartile is still big, so i decide to use only the 10% bigger

In [ ]:
v_5q  = df[(df['views'] > np.quantile(df['views'],0.90)) & (df['views'] <= np.quantile(df['views'],0.99))]
px.histogram(v_5q,x = 'views',nbins=90)

### Boxplot only for the 10%
* There are some categories that doesnt have extremaly outliders, maybe those videos have the same behaviour
* Im going to analyze the behaviour of ** Autos & Vehicles, Science & Tecnology, Education, Pets & animals and Travel & Events**
* i'm going to analyze MUSIC out of the group that i choosed
* Im excluing Otros because i dont know the  real category of that


In [ ]:
px.box(v_4q, y = 'views', x ='categoria' ) 

# Creating new df for categories that i choose
* Add %likes, %dislike and %comment in order to have better viz

In [ ]:
df_a = v_4q[v_4q['categoria'].isin(
    ['Autos & Vehicles','Science & Technology','Education','Pets & Animals','Travel & Events'])]

In [ ]:
df_a['p_likes'] = round((df_a['likes']/df_a['views'])*100,2)
df_a['p_dislikes'] = round((df_a['dislikes']/df_a['views'])*100,2)
df_a['p_comment'] = round((df_a['comment_count']/df_a['views'])*100,2)


## Top Videos

### Videos by Category
* Categories that i choosed before have less videos but big views impact

In [ ]:
df.groupby('categoria')['categoria'].agg('count').sort_values(ascending = False)

##### Videos by Channel - top 10

In [ ]:
df_a.groupby(
    'channel_title')['channel_title'].count().sort_values(ascending = False).head(10)

# Analyzing only categories selected

* Means of principal variables by category
* comment's percentage have the same behaviour in all categories
* dislike's percentage have the same behaviour in all categories
* categories with more views have less like's percentage

In [ ]:
df_a.groupby(
    'categoria')[['views','p_likes','p_dislikes','p_comment']].mean().round(1).sort_values('views', ascending = False) 

In [ ]:
px.histogram(df_a, x = 'p_comment',color = 'categoria')
px.histogram(df_a, x = 'p_dislikes',color = 'categoria')
px.histogram(df_a, x = 'p_likes',color = 'categoria')

In [ ]:
px.histogram(df_a, x = 'p_likes',facet_col = 'categoria',facet_col_wrap=1).update_yaxes(matches=None)

In [ ]:
px.scatter(df_a,'views','p_likes', facet_col="categoria",facet_col_wrap=5).update_yaxes(matches=None).update_xaxes(matches=None)

In [ ]:
px.scatter(df_a,'views','p_likes', color = 'categoria')

In [ ]:
df_a[df_a.ratings_disabled == True]